# Dataset

In [ ]:
import json
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sklearn.cross_validation import train_test_split
from sklearn import metrics
pd.set_option('display.max_columns', 500)


with open("credentials.json") as f:
    credentials = json.loads(f.read())
    
    host = credentials["host"]
    user = credentials["db_user"]
    password = credentials["db_pass"]
    db = credentials["db_name"]

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{db}")

df = pd.read_sql_query('SELECT * FROM trips_2017 WHERE lineid = "46A" AND direction = 1', engine)
df.head()

In [ ]:
# Replace missing actual time departure values with timetable values
df.actualtime_dep.fillna(df.plannedtime_dep, inplace=True)
df.head()

In [ ]:
# Remove rows with missing values for actual time arrival as we cannot safely assume these are as per timetable
df = df[pd.notnull(df['actualtime_arr'])]
df.head()

In [ ]:
# Create a new column for trip duration
df['trip_duration'] = df['actualtime_arr'] - df['actualtime_dep']
df.head()

In [ ]:
# Create a new column with the hour of the day the trip took place
df['actualtime_dep_H'] = round(df['actualtime_dep']/3600)
df.head()

In [ ]:
# Hour of actual time arrival
df['actualtime_arr_H'] = round(df['actualtime_arr']/3600)
df.head()

In [ ]:
# Average hour of the day of the journey
df['avg_H'] = (df['actualtime_dep_H'] + df['actualtime_arr_H']) / 2
df.head()

In [ ]:
df['avg_H'] = df['avg_H'].astype(int)
df.head()

In [ ]:
# Creating column solely for the dates to correlate with the dates column on the historical weather data table
df['time'] = df['timestamp'] + df['avg_H'] * 3600
df.time

In [ ]:
# Removing suppressed rows where suppressed=1.0
df = df.query('suppressed != 1.0')

In [ ]:
df.index = range(len(df))

In [ ]:
# Creating columns from timestamp for further processing
df['dayofweek'] = df['timestamp']
df['monthofyear'] = df['timestamp']

In [ ]:
# Converting the unix time to datetime format
df.dayofweek = pd.to_datetime(df['dayofweek'], unit='s')
df.monthofyear = pd.to_datetime(df['monthofyear'], unit='s')

In [ ]:
# Converting datetime to name of weekday, and to name of month (in separate columns)
df['dayofweek'] = df['dayofweek'].dt.weekday_name
df['monthofyear'] = df['monthofyear'].dt.month

In [ ]:
# Creating dummy variables for weekday names and name of month
df_dayofweek_dummies = pd.get_dummies(df['dayofweek'])


In [ ]:
# Removing rows not in the month of March
df = df.query('monthofyear == 3')

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df1 = pd.concat([df, df_dayofweek_dummies], axis=1, join_axes=[df.index])

In [ ]:
df1

In [ ]:
# Pull weather data from database
df2 = pd.read_sql_query('SELECT * FROM DarkSky_historical_weather_data WHERE year = 2017', engine)
df2.head()

In [ ]:
d = {'clear-day':'clear','clear-night':'clear','partly-cloudy-day':'partly-cloudy','partly-cloudy-night':'partly-cloudy'}
df2 = df2.replace(d)

In [ ]:
df2.rename(columns={'day_of_week': 'dayofweek', 'month': 'monthofyear'}, inplace=True)

In [ ]:
df3 = pd.merge(df1, df2, on=['time'])

In [ ]:
df3.head()

In [ ]:
df3 = df3[['avg_H', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'temp', 'precip_intensity','trip_duration']]

In [ ]:
# Trip duration is in seconds, convert to minutes and round to the nearest integer
df3['trip_duration'] = round(df3['trip_duration']/60)

In [ ]:
df3['trip_duration'] = df3['trip_duration'].astype(int)

In [ ]:
df3['temp'] = round(df3['temp'])

In [ ]:
df3['temp'] = df3['temp'].astype(int)

In [ ]:
#df3 = df3[['avg_H', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'temp','trip_duration']]

In [ ]:
df3.head()

In [ ]:
df3.shape

# Preprocessing
You can see that our dataset has eleven columns. The task is to predict the trip duration (last column) based on the day of the week, the time of the day and the weather conditions (temperature and rain intesity). The next step is to split our dataset into attributes and labels. 

In [ ]:
# Assign data from first four columns to X variable
X = df3.iloc[:, 0:10]

# Assign data from fifth column to y variable
y = df3['trip_duration']

In [ ]:
y.head()

In [ ]:
# Split the dataset 70/30
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30) 

# Random Forest Regression 
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#examples-using-sklearn-ensemble-randomforestregressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [ ]:
# default is 10 trees; use n_estimators : integer, optional (default=10)
# The number of trees in the forest.
regr = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=0)
regr.fit(X, y)

In [ ]:
import pickle
# storing the model trained on the full data set to a pickle file
pkl_filename = "RFR_March_2017_46A_1.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(regr, file)

In [ ]:
# Compute the importance of each feature based on the model
pd.DataFrame({'feature': X.columns, 'importance': regr.feature_importances_})

In [ ]:
# predict for 9 am on a Tuesday with 0.0 rain and 12 degrees
print(round(regr.predict([[9, 0, 1, 0, 0, 0, 0, 0, 12, 0.0]])[0]),"minutes")

In [ ]:
pred = regr.predict(X)

In [ ]:
predictions = pd.DataFrame(pred)
predictions.rename(columns={0:'estimated_time'}, inplace=True )
predictions['estimated_time'] = round(predictions['estimated_time'])
predictions['estimated_time'] = predictions['estimated_time'].astype(int)
predictions.head()

In [ ]:
print("Accuracy: ", metrics.accuracy_score(y, predictions))
print("Confusion matrix: \n", metrics.confusion_matrix(y, predictions))
print("Classification report:\n ", metrics.classification_report(y, predictions))


# Splitting data into Train and Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("Training data:\n", pd.concat([X_train, y_train], axis=1))
print("\nTest data:\n", pd.concat([X_test, y_test], axis=1))

In [ ]:
# Fit the data using the training set
#regr.fit(X_train, y_train)

In [ ]:
# Compute the importance of each feature based on the model
#pd.DataFrame({'feature': X.columns, 'importance': regr.feature_importances_})

In [ ]:
#rfc_predictions_test = regr.predict(X_test)
#df_true_vs_rfc_predicted_test = pd.DataFrame({'ActualClass': y_test, 'PredictedClass': rfc_predictions_test})
#df_true_vs_rfc_predicted_test

# Extracting pickle data

In [ ]:
# Load the pickle file
with open('RFR_March_2017_46A_1.pkl','rb') as picklefile:
        rfr_ = pickle.load(picklefile)

In [ ]:
# Generate current date and time
import datetime as dt
day = dt.datetime.today().weekday() # current day of the week 0=Monday
hour = (dt.datetime.now().hour)  # current hour 

In [ ]:
# Dataframe to hold current time
df_time = pd.DataFrame({'avg_H': [hour]})

In [ ]:
# Data frame to hold current day of week DOW
df_dummies_DOW = pd.DataFrame({'DOW_Monday': [1], 'DOW_Tuesday': [0],'DOW_Wednesday': [0], 'DOW_Thursday': [0],'DOW_Friday': [0], 'DOW_Saturday': [0],'DOW_Sunday': [0]})

if day == 0:
    df_dummies_DOW = pd.DataFrame({'DOW_Monday': [1], 'DOW_Tuesday': [0],'DOW_Wednesday': [0], 'DOW_Thursday': [0],'DOW_Friday': [0], 'DOW_Saturday': [0],'DOW_Sunday': [0]})
elif day == 1:
    df_dummies_DOW = pd.DataFrame({'DOW_Monday': [0], 'DOW_Tuesday': [1],'DOW_Wednesday': [0], 'DOW_Thursday': [0],'DOW_Friday': [0], 'DOW_Saturday': [0],'DOW_Sunday': [0]})
elif day == 2:
    df_dummies_DOW = pd.DataFrame({'DOW_Monday': [0], 'DOW_Tuesday': [0],'DOW_Wednesday': [1], 'DOW_Thursday': [0],'DOW_Friday': [0], 'DOW_Saturday': [0],'DOW_Sunday': [0]})
elif day == 3:
    df_dummies_DOW = pd.DataFrame({'DOW_Monday': [0], 'DOW_Tuesday': [0],'DOW_Wednesday': [0], 'DOW_Thursday': [1],'DOW_Friday': [0], 'DOW_Saturday': [0],'DOW_Sunday': [0]})
elif day == 4:
    df_dummies_DOW = pd.DataFrame({'DOW_Monday': [0], 'DOW_Tuesday': [0],'DOW_Wednesday': [0], 'DOW_Thursday': [0],'DOW_Friday': [1], 'DOW_Saturday': [0],'DOW_Sunday': [0]})
elif day == 5:
    df_dummies_DOW = pd.DataFrame({'DOW_Monday': [0], 'DOW_Tuesday': [0],'DOW_Wednesday': [0], 'DOW_Thursday': [0],'DOW_Friday': [0], 'DOW_Saturday': [1],'DOW_Sunday': [0]})
else:
    df_dummies_DOW = pd.DataFrame({'DOW_Monday': [0], 'DOW_Tuesday': [0],'DOW_Wednesday': [0], 'DOW_Thursday': [0],'DOW_Friday': [0], 'DOW_Saturday': [0],'DOW_Sunday': [1]})


In [ ]:
# Current weather call to database (uses latest timestamp)

df = pd.read_sql_query('SELECT * FROM DarkSky_current_weather ORDER BY timestamp DESC LIMIT 1;', engine)
df.head()

In [ ]:
# Dataframe to hold current temperature and rain
df_weather = pd.DataFrame({'temp': df['temp'], 'precip_intensity': df['precip_intensity']})

In [ ]:
# Put all our data in one place
dfX = pd.concat([df_time, df_dummies_DOW[['DOW_Monday', 'DOW_Tuesday','DOW_Wednesday', 'DOW_Thursday','DOW_Friday', 'DOW_Saturday','DOW_Sunday']],df_weather], axis =1)
dfX

In [ ]:
# Predict journey time (first to last stop) given local time and weather
print("Your journey will take around", int(round(rfr_.predict(dfX)[0])), "minutes.")

In [ ]:
total_number_of_stops = 65 # 65 based on leavetimes progress number
origin_stop = 10 # generated from user input based on progression number
destination_stop = 17
number_of_stops_travelled = destination_stop - origin_stop
percentage_of_route_travelled = (number_of_stops_travelled/(total_number_of_stops - 1)) * 100
print("That is %" + str(percentage_of_route_travelled) + " of the entire journey")

In [ ]:
total_time = int(round((int(round(rfr_.predict(dfX)[0])) * percentage_of_route_travelled) / 100))
print("If you wish to travel between stops", origin_stop, "and", destination_stop, "your journey will take approximately", total_time, "minutes.")

In [ ]:
# Current weather call to database (uses latest timestamp)

df_stops = pd.read_sql_query('SELECT * FROM static_bus_stop_data;', engine)
df_stops.head()

In [ ]:
# Generating a list of stops serviced by the 46A

df_lt = pd.read_sql_query('SELECT * FROM leavetimes_2017 WHERE tripid = 4395906;', engine)
df_lt.head()

In [ ]:
df_bus_stop_data = pd.DataFrame({'stop_id': df_stops['stop_id'], 'full_name': df_stops['full_name'],'lat': df_stops['latitude'], 'long': df_stops['longitude']})

In [ ]:
temp = pd.DataFrame({'progress': df_lt['progrnumber'], 'stop_id': df_lt['stoppointid']})

In [ ]:
merged = pd.merge(df_bus_stop_data, temp, on=['stop_id'])

In [ ]:
# merged dataframe contains every stop serviced (id, name and progress number) by the 46A ordered by progression number
merged = merged.drop_duplicates(keep='first')
merged = merged.sort_values(['progress'])
merged

In [ ]:
origin_stop = input("Enter origin stop: ")
destination_stop = input("Enter destination stop: ")

In [ ]:
merged.to_csv('46A_stops_data.csv', sep='\t', encoding='utf-8')

In [ ]:
# Forecast
day_ = dt.datetime.today().weekday() # current day of the week 0=Monday
hour_ = (dt.datetime.now().hour)  # current hour 

print(day_,hour_)

In [ ]:
for i in range(7):
    
    day_ += 1
    day_ = day_ % 7
    before = hour_ -1
    now = hour_
    after = hour_ +1
    print("The day is",day_,"and the hour is",now,before,after)